<a href="https://colab.research.google.com/github/JMassing/DeepL-CardDetection/blob/feature%2Fyolo_tiny/Yolo/Yolov4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yolov4 

Train Yolov4 on Synthetic Card images

Notebook follows tutorial on :https://colab.research.google.com/drive/12QusaaRj_lUwCGDvQNfICpa7kA7_a2dE#scrollTo=68eMertgIxaB

## Step 1 - Helper functions

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

## Step 2 - Download an build Yolov4

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# Download pretrained weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

## Step 3 - Check if Yolov4 runs correctly

In [ ]:
# run darknet detection on test images
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
# show image using our helper function
imShow('predictions.jpg')

## Step 4 - Train on cards object

In [ ]:
# Helper function to split data
def split_data(path, split):
  import os
  from sklearn.utils import shuffle
  if os.path.exists(path) is False:
    print("[Warning]: Path was not found")
    return [], []
  else:
    _, _, filenames = next(os.walk(path))
    filenames = [file for file in filenames if file.endswith('.jpg')]
    filenames = shuffle(filenames)
    idx = int(len(filenames) * split)
    train_files = filenames[0:idx]
    val_files = filenames [idx:]
    return train_files, val_files

In [ ]:
# unzip data
import zipfile
import os
path = os.path.join("build", "darknet", "x64", "data")
zip_ref = zipfile.ZipFile(os.path.join(path, "cards.zip"), 'r')
zip_ref.extractall(os.path.join(path, "cards"))

In [ ]:
# Split training data into validation and training data and write train.txt and
# test.txt required for yolo training
import os
path = os.path.join("build", "darknet", "x64", "data", "cards")
train_data, val_data = split_data(path, 0.8)

In [ ]:
# write train.txt and test.txt for yolo training
path = os.path.join("build", "darknet", "x64", "data")
cards_path = os.path.join("build", "darknet", "x64", "data", "cards")
with open(os.path.join(path, "train.txt"), "w") as fn:
  for img in train_data:
    line = f"{os.path.join(cards_path, img)}\n"
    fn.write(line)

with open(os.path.join(path, "test.txt"), "w") as fn:
  for img in val_data:
    line = f"{os.path.join(cards_path, img)}\n"
    fn.write(line)

In [ ]:
# get config files from repo and copy to darknet
%cd ../
!git clone https://github.com/JMassing/DeepL-CardDetection.git
%cd DeepL-CardDetection
!git checkout master
!cp Yolo/cards.data ../darknet/build/darknet/x64/data/
!cp Yolo/cards.names ../darknet/build/darknet/x64/data/
!cp Yolo/yolov4-cards.cfg ../darknet/build/darknet/x64/cfg/
!cp Yolo/yolov4-tiny-cards.cfg ../darknet/build/darknet/x64/cfg/
%cd ../darknet

In [ ]:
# helper function for training
def train_yolo(config = "default"):
  if config is "default":
    # download the weights
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
    # train the network
    !./darknet detector train build/darknet/x64/data/cards.data \
    build/darknet/x64/cfg/yolov4-cards.cfg yolov4.conv.137 -dont_show -map
  elif config is "tiny":
    # download the weights
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
    # train the network
    !./darknet detector train build/darknet/x64/data/cards.data \
    build/darknet/x64/cfg/yolov4-tiny-cards.cfg yolov4-tiny.conv.29 -dont_show -map
  else:
    print("Unknown Yolo Config")

def test_detection(config = "default"):
  if config is "default":
    # run darknet detection on test images
    !./darknet detector test build/darknet/x64/data/cards.data build/darknet/x64/cfg/yolov4-cards.cfg \
    yolov4-cards_1000.weights valid_data/image_00005.jpg
  elif config is "tiny":
    # run darknet detection on test images
    !./darknet detector test build/darknet/x64/data/cards.data build/darknet/x64/cfg/yolov4-tiny-cards.cfg \
    yolov4-tiny-cards_best.weights valid_data/image_00005.jpg
  else:
    print("Unknown Yolo Config")

In [ ]:
# train yolo 
config = "tiny"
train_yolo(config = config)

In [ ]:
# test detection on image
test_detection(config)
# show image using our helper function
imShow('predictions.jpg')